## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
path_file = os.path.join('../Weather_Database', 'WeatherPy_Database.csv')
city_data_df = pd.read_csv(path_file)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Punta Arenas,CL,-53.1500,-70.9167,57.31,58.0,0.0,4.61,clear sky
1,Tocopilla,CL,-22.0920,-70.1979,70.77,79.0,31.0,9.89,scattered clouds
2,Kodiak,US,57.7900,-152.4072,38.71,93.0,100.0,24.16,moderate rain
3,Rikitea,PF,-23.1203,-134.9692,78.82,75.0,100.0,7.94,overcast clouds
4,Madimba,TZ,-10.4000,40.3333,74.77,91.0,100.0,6.53,light rain


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &\
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Punta Arenas,CL,-53.1500,-70.9167,57.31,58.0,0.0,4.61,clear sky
13,Ponta Do Sol,PT,32.6667,-17.1000,65.03,81.0,88.0,15.43,overcast clouds
15,Airai,TL,-8.9266,125.4092,61.47,98.0,100.0,3.74,overcast clouds
26,Hermanus,ZA,-34.4187,19.2345,60.91,87.0,2.0,10.00,clear sky
29,Cape Town,ZA,-33.9258,18.4232,64.11,82.0,0.0,4.00,clear sky
35,Jiangkou,CN,25.4872,119.1986,51.69,74.0,96.0,14.79,overcast clouds
36,Bredasdorp,ZA,-34.5322,20.0403,58.28,87.0,0.0,1.14,clear sky
38,Praia Da Vitoria,PT,38.7333,-27.0667,61.75,94.0,20.0,17.05,few clouds
39,Mataura,NZ,-46.1927,168.8643,58.48,57.0,92.0,8.75,overcast clouds
41,Ribeira Brava,PT,32.6500,-17.0667,65.03,83.0,86.0,16.13,overcast clouds


In [14]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

143


City                   143
Country                140
Lat                    143
Lng                    143
Max Temp               143
Humidity               143
Cloudiness             143
Wind Speed             143
Current Description    143
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
print(clean_df.count())

City                   140
Country                140
Lat                    140
Lng                    140
Max Temp               140
Humidity               140
Cloudiness             140
Wind Speed             140
Current Description    140
dtype: int64


In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,57.31,clear sky,-53.1500,-70.9167,
13,Ponta Do Sol,PT,65.03,overcast clouds,32.6667,-17.1000,
15,Airai,TL,61.47,overcast clouds,-8.9266,125.4092,
26,Hermanus,ZA,60.91,clear sky,-34.4187,19.2345,
29,Cape Town,ZA,64.11,clear sky,-33.9258,18.4232,
35,Jiangkou,CN,51.69,overcast clouds,25.4872,119.1986,
36,Bredasdorp,ZA,58.28,clear sky,-34.5322,20.0403,
38,Praia Da Vitoria,PT,61.75,few clouds,38.7333,-27.0667,
39,Mataura,NZ,58.48,overcast clouds,-46.1927,168.8643,
41,Ribeira Brava,PT,65.03,overcast clouds,32.6500,-17.0667,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for row in clean_df.itertuples():
    # 6c. Get latitude and longitude from DataFrame
    lat = row[3]
    long = row[4]
    params["location"] = f"{lat},{long}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[row[0], "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
print(hotel_df.count())
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
print(clean_hotel_df.count())
clean_hotel_df.head()

140
City                   140
Country                140
Max Temp               140
Current Description    140
Lat                    140
Lng                    140
Hotel Name             140
dtype: int64
City                   125
Country                125
Max Temp               125
Current Description    125
Lat                    125
Lng                    125
Hotel Name             125
dtype: int64


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,57.31,clear sky,-53.1500,-70.9167,Hotel Hain
13,Ponta Do Sol,PT,65.03,overcast clouds,32.6667,-17.1000,Hotel do Campo
26,Hermanus,ZA,60.91,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
29,Cape Town,ZA,64.11,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
35,Jiangkou,CN,51.69,overcast clouds,25.4872,119.1986,DFM Hotel


In [44]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))